In [1]:
import os
import sys
sys.path.append(os.chdir('../../'))

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.plotly as ply
import plotly.graph_objs as go
import src.utils.graph as graph
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

#from sklearn.multioutput import MultiOutputRegressor
from src.algorithms.multioutput import MultiOutputRegressor, RegressorChain

from sklearn.utils import shuffle
#from catboost import CatBoostRegressor
from src.algorithms.catboost_model import CatBoost

from sklearn.metrics import mean_absolute_error

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

import src.data as data
import src.utility as utility

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



## Baseline predicting the last available speed

In [3]:
mode = 'full'
t = 'train'
d = data.base_dataset(mode, t) #, onehot=False, drop_index_columns=True)
d.shape

caching resources/dataset/preprocessed/full/train/base_dataset.csv.gz


(27545, 62)

#### Taking the last 20% as validation

In [4]:
val_split = int(d.shape[0] * 0.2)
d = d.sort_values('DATETIME_UTC_y_0').iloc[-val_split:]
d.shape

(5509, 62)

#### Dropping rows with all Y and X NaN

In [5]:
col_x = ['SPEED_AVG_-4','SPEED_AVG_-3','SPEED_AVG_-2','SPEED_AVG_-1']
col_y = ['SPEED_AVG_Y_0','SPEED_AVG_Y_1','SPEED_AVG_Y_2','SPEED_AVG_Y_3']
m = d[col_x + col_y]
m.dropna(subset=col_y, how='all', inplace=True)
print('After droppping when all Y are NaN:', m.shape)
m.dropna(subset=col_x, how='all', inplace=True)
print('After droppping when all X are NaN:', m.shape)

After droppping when all Y are NaN: (3016, 8)
After droppping when all X are NaN: (2879, 8)


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [6]:
print(1354 / 3034)

0.44627554383651946


In [7]:
m.head()

,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_AVG_Y_0,SPEED_AVG_Y_1,SPEED_AVG_Y_2,SPEED_AVG_Y_3
20951,173.013354,175.965019,175.564315,177.629711,177.077397,172.846586,176.122042,174.254342
25328,137.634387,139.640435,139.965010,138.147175,134.951868,137.810038,133.129040,144.627627
26449,176.567460,177.469053,182.889793,180.575542,179.780025,177.980000,175.561890,178.495626
21900,198.793364,196.349059,194.136886,191.733000,182.968833,190.614170,183.329839,198.428766
22151,194.904610,194.958740,190.150038,198.053749,197.229881,189.575667,185.694545,196.016825


In [8]:
# fill nan in x, so that the prediction is the first not NaN speed avg before target
mask_nan = m['SPEED_AVG_-1'].isnull()
print(sum(mask_nan*1))

125


In [9]:
m[mask_nan].head(3)

,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_AVG_Y_0,SPEED_AVG_Y_1,SPEED_AVG_Y_2,SPEED_AVG_Y_3
20654,221.666,188.270480,199.823,NaN,197.036444,210.474833,175.258818,NaN
24739,167.463,167.732667,150.474,NaN,NaN,161.395500,NaN,NaN
25339,NaN,119.732000,NaN,NaN,NaN,NaN,127.822000,NaN


In [10]:
m[col_x] = m[col_x].fillna(method='ffill', axis=1)
print(sum(m['SPEED_AVG_-1'].isnull()*1))

0


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



#### Cleaned dataset

In [11]:
m.head()

,SPEED_AVG_-4,SPEED_AVG_-3,SPEED_AVG_-2,SPEED_AVG_-1,SPEED_AVG_Y_0,SPEED_AVG_Y_1,SPEED_AVG_Y_2,SPEED_AVG_Y_3
20951,173.013354,175.965019,175.564315,177.629711,177.077397,172.846586,176.122042,174.254342
25328,137.634387,139.640435,139.965010,138.147175,134.951868,137.810038,133.129040,144.627627
26449,176.567460,177.469053,182.889793,180.575542,179.780025,177.980000,175.561890,178.495626
21900,198.793364,196.349059,194.136886,191.733000,182.968833,190.614170,183.329839,198.428766
22151,194.904610,194.958740,190.150038,198.053749,197.229881,189.575667,185.694545,196.016825


In [12]:
mae_4 = []
for col in col_y:
    # mae for each target
    diff = np.abs(m['SPEED_AVG_-1'] - m[col])
    mae_4.append(np.mean(diff))

In [13]:
mae_4

[7.736181997473435, 8.85876905995962, 9.848053663420394, 10.307130125042415]

In [14]:
np.mean(mae_4)

9.187533711473966